In [11]:
## Load Folder Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive/')

## Move Pointer File ke DATA_UM
%cd /content/drive/MyDrive/DATA_UM

## Declare Root Project File
root_file = "/content/drive/MyDrive/DATA_UM"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/DATA_UM


In [12]:
import numpy as np
import pandas as pd

import re
import string

import os.path
from os import path

In [13]:
## Load Preprocess SLANG Module

# Load Dataframe
df_tmp_nasalsabila = pd.read_csv('data/slang/colloquial-indonesian-lexicon_nasalsabila.csv')
df_slang_fendiirfan = pd.read_csv('data/slang/colloquial-indonesian-lexicon_fendiirfan.csv')
df_slang_custom = pd.read_csv('data/slang/custom_dictonary_slang.csv')
df_slang_custom_exception = pd.read_csv('data/slang/custom_dictonary_slang_dont_swap.csv')
df_slang_custom_stemming = pd.read_csv('data/slang/custom_dictonary_slang_stemming.csv')

# Transform and Delete Duplicate Data from nasalsabila
df_slang_nasalsabila = df_tmp_nasalsabila[['slang', 'formal']]
df_slang_nasalsabila.drop_duplicates(inplace=True)

# Transform and Delete Duplicate Data from fendiirfan
df_slang_fendiirfan.dropna(inplace=True)
df_slang_fendiirfan.rename(columns={'kataAlay': 'slang', 'kataBaik': 'formal'}, inplace=True)
df_slang_fendiirfan.drop_duplicates(inplace=True)

# Transform and Delete Duplicate our own data.
df_slang_custom.drop_duplicates(inplace=True)
df_slang_custom_exception.drop_duplicates(inplace=True)
df_slang_custom_stemming.drop_duplicates(inplace=True)

def remove_slang(text, df_tmp):
    # To Lowercase
    text = text.lower()

    # Pecah to array
    words = text.split(" ");

    # DF to Array
    arr_slang = df_tmp['slang'].to_numpy()
    arr_formal = df_tmp['formal'].to_numpy()
    do_not_change = df_slang_custom_exception['kata'].to_numpy()

    # Array New Text
    new_text = []

    for kata in words:
      if kata in arr_slang:
        if kata not in do_not_change:
          idx = np.where(arr_slang==kata)[0][0]
          kata = arr_formal[idx]

      new_text.append(kata)
      text_new = ' '.join(new_text)

    return text_new

<ipython-input-13-4e114e166bee>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slang_nasalsabila.drop_duplicates(inplace=True)


In [14]:
## Load Preprocess Stemming Modul
# Load and Download Library Sastrawi
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Indonesian Stopword by Sastrawi
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [15]:
## Load Preprocess Stopword Data

# Load Data Stopword from Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# Load Data Stopword and Custom Word
stopword_satya = list(pd.read_csv('data/stopword/stopwords_satya.txt', header = None)[0])
custom_stopword = list(pd.read_csv('data/stopword/custom_stopword.txt', header = None)[0])
keep_custom_stopword = list(pd.read_csv('data/stopword/keep_custom_stopword.txt', header = None)[0])

# Declare Function / Modul Stopword
def remove_stopwords(text):
  # Removal Stopword
  text_split = text.split(' ')

  temp_text_split = []
  for i in range(len(text_split)):
    ## Remove Stopword by Satwa : gak dipake dulu ini terlalu ngilangin konteks text
    #if (text_split[i] not in stopword_satya) and (text_split[i] not in custom_stopword):
    if (text_split[i] not in custom_stopword):
      ## Keep Beberapa Kata Valence of a text
      # https://datascience.stackexchange.com/questions/15765/nlp-why-is-not-a-stop-word#:~:text=Stop%20words%20are%20usually%20thought,idf%20analysis%20for%20document%20classification).
      if (text_split[i] in keep_custom_stopword) :
        temp_text_split.append(text_split[i])
      else :
        ## Stopword Removal from data by Sastrawi
        tmp = stopword.remove(text_split[i])
        temp_text_split.append(tmp)

  final_text = ' '.join(temp_text_split)

  # will replace single word
  final_text = re.sub(r"\b[a-zA-Z]\b","",final_text)

  # will replace space more than one
  final_text = re.sub("( {2,})"," ",final_text)

  return final_text

In [16]:
def clean_text_predict(text) :

    text = str(text)

    ## TEXT TRANSFORM
    # Replace TAB with space
    text = text.replace('\t', ' ')

    # Remove username 1st
    text = re.sub('@[^\s]+','',text)

    # Remove hashtag dan @
    #''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

    # Change "-" to space
    text = text.replace("-", " ")

    # Remove punctiation wihtout underscore
    pattern = r'[^\w\s\/\']'
    text = re.sub(pattern, ' ', text)
    text = text.replace("_", " ")
    # translator = re.compile('[%s]' % re.escape('!"#$%&\'()*+,-.:;<=>?@[\\]^_`{|}~'))
    # translator.sub(' ', text)

    # Save Special Character
    text = text.replace("/", " bimzslash ")

    # Remove angka
    #text = re.sub('[0-9]+', '', text)

    # Remove website link
    text = re.sub(r'http\S+', '', text)

    # Remove new line
    text = re.sub("\n"," ",text)

    # Transform to lowercase
    text = text.lower()

    # Slang change for spesific case (gg, dll yang menghapus multi same char)
    text = remove_slang(text ,df_slang_custom)

    # Replace perulangan huruf pada kalimat "iyaaa" to "iya"
    #text = re.sub(r'(\w)(\1{2,})', r"\1", text) #removing number and letters
    text = re.sub(r'([a-zA-Z])(\1{2,})', r"\1", text) #only remove number

    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)

    # will replace space more than one
    text = re.sub('( {2,})',' ',text)

    ## SLANG
    text = remove_slang(text ,df_slang_custom)
    text = remove_slang(text ,df_slang_nasalsabila)
    text = remove_slang(text ,df_slang_fendiirfan)

    ## STEMMING
    text = stemmer.stem(text)

    ## STOPWORD
    text = remove_stopwords(text)

    ## ROLL BACK STEMMING
    text = remove_slang(text ,df_slang_custom_stemming)

    ## TEXT TRANSFORM BUT AFTER
    # will replace single word
    # text = re.sub(r"\b[a-zA-Z]\b","",text)

    return text

In [18]:
df_clean = pd.read_excel(open('data/training/ultramilk_all_data/test_new/master_data_remove_duplicate.xlsx', 'rb'), sheet_name='dataset')
df_clean = df_clean.loc[:, ~df_clean.columns.str.contains('^Unnamed')]
df_clean.drop(columns=df_clean.columns[0], axis=1, inplace=True)
df_clean.reset_index(drop=True, inplace=True)
df_clean

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,revisi,text_final,not_balanced,balance
0,2023-10-27,Twitter,ultramilk,iamultraoddinary,Make Your Move!,@ultramym,1197372091978276864,279.0,11687.0,0.0,933.0,49556,https://twitter.com/ultramym/status/1717890328...,Dengan bangga mempersembahkan @Stray_Kids seba...,4,4,bangga sembah bagai brand ambassador baru ult...,Positive,Positive
1,2023-11-01,Twitter,ultramilk,iamultraoddinary,Make Your Move!,@ultramym,1197372091978276864,526.0,11012.0,0.0,2015.0,39385,https://twitter.com/ultramym/status/1719564927...,Gimana ULTRAoddinary versi @Stray_Kids bareng ...,4,4,bagaimana ultraoddinary versi bareng ultra mil...,Neutral,Positive
2,2023-11-01,Twitter,ultramilk,makeyourmove,Make Your Move!,@ultramym,1197372091978276864,526.0,11012.0,0.0,2015.0,39385,https://twitter.com/ultramym/status/1719564927...,Gimana ULTRAoddinary versi @Stray_Kids bareng ...,4,4,bagaimana ultraoddinary versi bareng ultra mil...,Neutral,Positive
3,2022-07-08,Twitter,ultramilk,dontstopyourmove,Make Your Move!,@ultramym,1197372091978276864,329.0,2862.0,0.0,1057.0,13633,https://twitter.com/ultramym/status/1545211062...,Taraaaaa~ Ultra Milk dan @ITZYofficial mengaja...,4,4,tara ultra milk ajak kamu buat jadi theunstopp...,Neutral,Neutral
4,2022-07-08,Twitter,ultramilk,theunstoppablegeneration,Make Your Move!,@ultramym,1197372091978276864,329.0,2862.0,0.0,1057.0,13633,https://twitter.com/ultramym/status/1545211062...,Taraaaaa~ Ultra Milk dan @ITZYofficial mengaja...,4,4,tara ultra milk ajak kamu buat jadi theunstopp...,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15608,2023-02-15 00:00:00,Instagram,Manipulation,Manipulation,👻,ciss_cake.st,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cor3xXEjoe-,Takut😭😭😭,0,0,takut,Negative,Negative
15609,2023-05-29 00:00:00,Instagram,Manipulation,Manipulation,Muhammad Hunain,wyvrnain,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/CsvXCxNO1Tu/,Lemah,0,0,lemah,Negative,Negative
15610,2023-05-27 00:00:00,Instagram,Manipulation,Manipulation,vinnn,kevinnn.setiawan666,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/CsvXCxNO1Tu/,Lemah,0,0,lemah,Negative,Negative
15611,2023-10-30 00:00:00,Instagram,Manipulation,Manipulation,abdul gani,abdul6185gani,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/CzBawcfPzGw/,Nakal,0,0,nakal,Negative,Negative


In [20]:
df_clean = df_clean.drop_duplicates(subset='text', keep="last")
df_clean.reset_index(drop=True, inplace=True)
df_clean

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,revisi,text_final,not_balanced,balance
0,2023-10-27,Twitter,ultramilk,iamultraoddinary,Make Your Move!,@ultramym,1197372091978276864,279.0,11687.0,0.0,933.0,49556,https://twitter.com/ultramym/status/1717890328...,Dengan bangga mempersembahkan @Stray_Kids seba...,4,4,bangga sembah bagai brand ambassador baru ult...,Positive,Positive
2,2023-11-01,Twitter,ultramilk,makeyourmove,Make Your Move!,@ultramym,1197372091978276864,526.0,11012.0,0.0,2015.0,39385,https://twitter.com/ultramym/status/1719564927...,Gimana ULTRAoddinary versi @Stray_Kids bareng ...,4,4,bagaimana ultraoddinary versi bareng ultra mil...,Neutral,Positive
4,2022-07-08,Twitter,ultramilk,theunstoppablegeneration,Make Your Move!,@ultramym,1197372091978276864,329.0,2862.0,0.0,1057.0,13633,https://twitter.com/ultramym/status/1545211062...,Taraaaaa~ Ultra Milk dan @ITZYofficial mengaja...,4,4,tara ultra milk ajak kamu buat jadi theunstopp...,Neutral,Neutral
5,2023-10-27,Twitter,ultramilk,iamultraoddinary,Make Your Move!,@ultramym,1197372091978276864,299.0,5290.0,0.0,826.0,12742,https://twitter.com/ultramym/status/1717753893...,Ada bocoran lagi nih! Udah makin gampang keteb...,4,4,bocor makin gampang tebak tahu jawab ikut teb...,Neutral,Neutral
7,2022-10-01,Twitter,ultramilk,theunstoppablegeneration,Make Your Move!,@ultramym,1197372091978276864,364.0,919.0,0.0,722.0,10235,https://twitter.com/ultramym/status/1576108978...,"WE HEAR YOU, K-MOVERS! Photocard @ultramym & ...",4,4,aku hear you movers photocard kamu indomaret a...,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15607,2023-02-15 00:00:00,Instagram,Manipulation,Manipulation,laela oktaviani,oktavianilaela,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cor3xXEjoe-,Najis😩,0,0,najis,Neutral,Negative
15608,2023-02-15 00:00:00,Instagram,Manipulation,Manipulation,👻,ciss_cake.st,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cor3xXEjoe-,Takut😭😭😭,0,0,takut,Negative,Negative
15610,2023-05-27 00:00:00,Instagram,Manipulation,Manipulation,vinnn,kevinnn.setiawan666,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/CsvXCxNO1Tu/,Lemah,0,0,lemah,Negative,Negative
15611,2023-10-30 00:00:00,Instagram,Manipulation,Manipulation,abdul gani,abdul6185gani,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/CzBawcfPzGw/,Nakal,0,0,nakal,Negative,Negative


In [21]:
df_clean['new_prepo'] = df_clean['text'].apply(lambda x: clean_text_predict(x))

<ipython-input-21-1f848ff3358f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['new_prepo'] = df_clean['text'].apply(lambda x: clean_text_predict(x))


In [22]:
df_clean.to_excel('data/training/ultramilk_all_data/test_new/remove_duplicate.xlsx')

In [23]:
df_clean = df_clean.drop_duplicates(subset='new_prepo', keep="last")
df_clean.reset_index(drop=True, inplace=True)
df_clean

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,labelling,revisi,text_final,not_balanced,balance,new_prepo
0,2023-11-01,Twitter,ultramilk,makeyourmove,Make Your Move!,@ultramym,1197372091978276864,526.0,11012.0,0.0,2015.0,39385,https://twitter.com/ultramym/status/1719564927...,Gimana ULTRAoddinary versi @Stray_Kids bareng ...,4,4,bagaimana ultraoddinary versi bareng ultra mil...,Neutral,Positive,bagaimana ultraoddinary versi bareng ultra mil...
1,2022-07-08,Twitter,ultramilk,theunstoppablegeneration,Make Your Move!,@ultramym,1197372091978276864,329.0,2862.0,0.0,1057.0,13633,https://twitter.com/ultramym/status/1545211062...,Taraaaaa~ Ultra Milk dan @ITZYofficial mengaja...,4,4,tara ultra milk ajak kamu buat jadi theunstopp...,Neutral,Neutral,tara ultra milk ajak kamu buat jadi theunstopp...
2,2023-10-27,Twitter,ultramilk,iamultraoddinary,Make Your Move!,@ultramym,1197372091978276864,299.0,5290.0,0.0,826.0,12742,https://twitter.com/ultramym/status/1717753893...,Ada bocoran lagi nih! Udah makin gampang keteb...,4,4,bocor makin gampang tebak tahu jawab ikut teb...,Neutral,Neutral,ada bocor sudah makin gampang tebak belum suda...
3,2022-10-01,Twitter,ultramilk,theunstoppablegeneration,Make Your Move!,@ultramym,1197372091978276864,364.0,919.0,0.0,722.0,10235,https://twitter.com/ultramym/status/1576108978...,"WE HEAR YOU, K-MOVERS! Photocard @ultramym & ...",4,4,aku hear you movers photocard kamu indomaret a...,Neutral,Neutral,aku hear you movers photocard sudah kamu dapat...
4,2023-10-27,Twitter,ultramilk,iamultraoddinary,Make Your Move!,@ultramym,1197372091978276864,179.0,3685.0,0.0,347.0,10004,https://twitter.com/ultramym/status/1717845402...,"Siap-siap buat liat dan langsung terpikat, kar...",4,4,siap siap buat lihat langsung pikat ultra milk...,Neutral,Positive,siap siap buat lihat langsung pikat ultra milk...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14032,2023-02-15 00:00:00,Instagram,Manipulation,Manipulation,laela oktaviani,oktavianilaela,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cor3xXEjoe-,Najis😩,0,0,najis,Neutral,Negative,najis
14033,2023-02-15 00:00:00,Instagram,Manipulation,Manipulation,👻,ciss_cake.st,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/Cor3xXEjoe-,Takut😭😭😭,0,0,takut,Negative,Negative,takut
14034,2023-05-27 00:00:00,Instagram,Manipulation,Manipulation,vinnn,kevinnn.setiawan666,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/CsvXCxNO1Tu/,Lemah,0,0,lemah,Negative,Negative,lemah
14035,2023-10-30 00:00:00,Instagram,Manipulation,Manipulation,abdul gani,abdul6185gani,Nan,NaN,NaN,NaN,NaN,0,https://www.instagram.com/p/CzBawcfPzGw/,Nakal,0,0,nakal,Negative,Negative,nakal


In [24]:
df_clean.to_excel('data/training/ultramilk_all_data/test_new/remove_duplicate_after_preproc.xlsx')